# การติดตั้งและเริ่มใช้งาน Hyperledger Fabric เบื้องต้น

*** ตัวอย่างนี้เป็นการติดตั้งและทดลองรัน Hyperledger Fabric version อย่างง่าย ซึ่งไม่เหมาะแ่กการใช้งานใน production environment ***

### Requirements

1. ติดตั้งโดยเครื่องมือ สคริปต์ และตัวอย่างที่ทาง official Hyperledger มีให้
2. ติดตั้งบนเครื่อง Stand alone เพียงเครื่องเดียว
3. ติดตั้งโดยใช้ Docker เป็นหลัก
4. ใช้ตัวอย่าง chaincode example_02 ที่ official มีให้
5. ผู้ติดตั้งมีสิทธิ์รัน docker

### System requirements

1. OS: Ubuntu 18.04 LTS (MacOS/Windows ก็สามารถติดตั้งได้เช่นกัน แต่ต้องใช้ทักษะ config Docker เพิ่มเติม)
2. CPU: ยิ่งเยอะยิ่งดี เพราะ Blockchain มี service หลายตัวมาก รวมไปถึงใช้งาน database หลาย instance
3. RAM: อย่างน้อย 8 GB สำหรับการติดตั้งแบบเครื่องเดียว
4. Internet: ไม่เช่นนั้นจะไม่สามารถ run docker container ได้

### Step โดยรวมในตัวอย่างนี้ประกอบด้วย

0. Setup เครื่องสำหรับใช้งาน notebook นี้
1. Setup เครื่องสำหรับการติดตั้งและใช้งาน Hyperledger Fabric
2. Generate crypto materail ซึ่งเป็น artifact สำหรับการสร้าง blockchain network และ permissioned channel สำหรับการทำ transaction  ต่างๆ
3. deploy blockchain network และทดสอบด้วย end to end test ที่ตัวอย่างให้มา
4. terminate blockchain service 

สำหรับเครื่องที่ใช้ทดสอบตัวอย่างนี้มีสเปคดังนี้

In [1]:
lsb_release -a
lscpu | grep "Model name"
lsmem | grep "Total online memory"

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.4 LTS
Release:	18.04
Codename:	bionic
Model name:          Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz
Total online memory:    15.9G


## Step 0 Install & Setup environment สำหรับ notebook นี้

*** step นี้ให้ run แต่ละ command ใน terminal นะครับ ***

ติดตั้ง  [Bash kernel](https://github.com/takluyver/bash_kernel) สำหรับ Jupyter จะได้ run Ubuntu command ได้จากใน notebook

```bash
pip install bash_kernel
python -m bash_kernel.install
```

แก้ให้ไม่ต้องใส่ password สำหรับคำสั่ง sudo ด้วยการ edit file /etc/sudoers

```shell
sudo visudo
```

ใส่

```text
username     ALL=(ALL) NOPASSWD:ALL
```

เข้าไปในบรรทัดสุดท้ายและ save file /etc/sudoers

เปิด notebook นี้ด้วยการรันคำสั่งใน terminal

```shell
jupyter notebook
```

## Step 1 Setup เครื่องที่ใช้งาน

### ติดตั้ง docker engine และ docker-compose ต่างๆ

ในบทความนี้จะใช้ Docker Commutnity Edition (Docker CE) เป็นหลักนะครับ

In [ ]:
sudo apt-get update

sudo apt-get install -y \
    apt-transport-https \
    ca-certificates \
    curl \
    gnupg-agent \
    software-properties-common \
    tree \
    zip unzip 
    
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
   $(lsb_release -cs) \
   stable"
   
sudo apt-get update

sudo apt-get install docker-ce docker-ce-cli containerd.io docker-compose

### ตั้งค่าให้ user สามารถใช้ docker ได้โดยไม่ต้องใช้ `sudo` ([Docker post installation](https://docs.docker.com/install/linux/linux-postinstall/))


In [ ]:
sudo groupadd docker
sudo usermod -aG docker $USER

### ทดสอบการทำงานของ docker

In [ ]:
docker run hello-world

## Step 2 Download Hyperledger Fabric binary files และ ตัวอย่าง

download file ที่จำเป็นในการใช้งาน โดยจะมีสิ่งที่ต้อง download 3 ประเภทหลัก คือ 

### 1. Hyperledger Fabric docker images
เป็น docker image ที่ pack core service ของ Hyperledger Fabric เอาไว้ ซึ่งสามารถ custom config/parameter ต่างๆได้
### 2. Hyperledger Fabric samples 
เป็นตัวอย่าง config และ chaincode (smart contract) ที่ทางทีมงาน Hyperledger เตรีมไว้ให้ทดลองเล่น และสามารถนำไปประยุกต์ใช้กับ app ของเราได้
### 3. Hyperledger Fabric platform-specific binaries
เป็น tool สำหรับการสร้าง crypto material และเรียกใช้งาน blockchain service ต่างๆ

คำสั่งที่ใช้ download ทุกอย่างแบบปกติ คือ

```shell
curl -sSL http://bit.ly/2ysbOFE | bash -s
```

step ข้างบนจะใช้เวลาค่อนข้างนาน เพราะต้อง download
1. Hyperledger Fabric docker images จำนวนมาก
2. Hyperledger Fabric samples หลายตัวอย่าง
3. Hyperledger Fabric platform-specific binaries ซึ่งใช้เวลา download ค่อนข้างนาน


เรามามารถ bypass ได้โดยการใส่ option

-d : bypass docker image download

-s : bypass fabric-samples repo clone

-b : bypass download of platform-specific binaries

ตัวอย่างการใช้งานเมื่อไม่ต้องการ download docker image และ samples

```shell
curl -sSL http://bit.ly/2ysbOFE | bash -s -- -d -s
```

ในตัวอย่างนี้จะทำการ download เฉพาะ binary file และตัวอย่างในเวอร์ชั่นเก่าเท่านั้น ซึ่ง docker image ที่จำเป็นจะถูก pull ในขณะ deploy service อยู่แล้ว

In [ ]:
# official scripts,
curl -sSL http://bit.ly/2ysbOFE | bash -s -- -d 1.4.7 1.4.7 0.4.15

[ข้อมูลการติดตั้งเพิ่มเติม](https://hyperledger-fabric.readthedocs.io/en/release-1.4/install.html)

Set system path เพื่อให้ระบบมองเห็น binary

In [ ]:
export PATH=$PWD/bin:$PATH

## Step 3 Generate crypto material

change directory ไปที่ fabric-samples/first-network

In [ ]:
cd fabric-samples/first-network
git checkout v1.4.7

ลองดูว่าใน script first network สามารถทำอะไรได้บ้าง

In [ ]:
bash byfn.sh

Generate crypto material สำหรับ channel ชื่อ `getstartfabric`

In [ ]:
# enter y to continue
echo "y" | 
bash byfn.sh generate -c getstartfabric

ตรวจสอบไฟล์ที่ script gen มาให้

In [ ]:
tree ./crypto-config

In [ ]:
tree ./channel-artifacts

## Step 3 Deploy Hyperledger Fabric Network

clean docker volume & image ก่อนทำการ deploy service

In [ ]:
docker kill $(docker ps -q -f name=example.com) $(docker ps -q -f name=cli)
docker system prune -f
docker volume prune -f

In [ ]:
echo "y" | 
bash ./byfn.sh up -c getstartfabric -i 1.4.7

ตรวจสอบการ deploy ว่าสำเร็จหรือไม่จาก log ที่ออกมา

ตรวจสอบ service ที่ถูก deploy ว่ามีอะไรบ้าง

In [ ]:
docker ps

## Step 4 Terminate Hyperledger Fabric Network

ทำการ shutdown และ clean docker container ต่างๆด้วย

**ข้อควรระวัง คำสั่งนี้จะทำการ kill docker container ทั้งหมดในเครื่อง**

In [ ]:
echo "y" |
bash ./byfn.sh down -c getstartfabric